# Transformer 模型


## pipeline 函数

🤗 Transformers 库中最基本的对象是 pipeline() 函数。它将模型与其必要的预处理和后处理步骤连接起来，使我们能够通过直接输入任何文本并获得最终的答案


In [ ]:
from transformers import pipeline

# 默认情况下，此pipeline选择一个特定的预训练模型，该模型已针对英语情感分析进行了微调。创建分类器对象时，将下载并缓存模型。如果您重新运行该命令，则将使用缓存的模型，无需再次下载模型。
classifier = pipeline("sentiment-analysis")  # 仅适合英文

In [7]:
classifier("I'm happy today!")

[{'label': 'POSITIVE', 'score': 0.9998737573623657}]

> 将一些文本传递到 pipeline 时涉及三个主要步骤：

> 1. 文本被预处理为模型可以理解的格式。
>
> 2. 预处理的输入被传递给模型。
>
> 3. 模型处理后输出最终人类可以理解的结果。

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)


### 使用分词器进行预处理(Tokenizer)

与其他神经网络一样，Transformer 模型无法直接处理原始文本， 因此我们管道的第一步是将文本输入转换为模型能够理解的数字。 为此，我们使用 tokenizer(标记器)，负责：

* 将输入拆分为单词、子单词或符号（如标点符号），称为标记(token)
* 将每个标记(token)映射到一个整数
* 添加可能对模型有用的其他输入

所有这些预处理都需要以与模型预训练时完全相同的方式完成，因此我们首先需要从 Model Hub 中下载这些信息。为此，我们使用 `AutoTokenizer` 类及其 `from_pretrained()` 方法。

使用我们模型的检查点名称，它将自动获取与模型的标记器相关联的数据，并对其进行缓存。


In [9]:
from transformers import AutoTokenizer

# sentiment-analysis（情绪分析）任务的预训练模型
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

一旦我们有了标记器(tokenizer)，我们就可以直接将我们的句子传递给它，然后我们就会得到一本字典，它可以提供给我们的模型！剩下要做的唯一一件事就是将输入 ID 列表转换为张量(tensor)。

**张量(tensor)**，你可以把它们想象成 NumPy 数组。NumPy 数组可以是标量（0D）、向量（1D）、矩阵（2D）或具有更多维度。它实际上是张量；其他 ML 框架的张量行为类似，通常与 NumPy 数组一样易于实例化。

要指定要返回的张量类型（PyTorch、TensorFlow 或 plain NumPy），我们使用 `return_tensors` 参数：


In [10]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True,
                   truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


输出本身是一个包含两个键的字典， `input_ids` 和 `attention_mask` 。 `input_ids` 包含两行整数（每个句子一行），它们是每个句子中标记的唯一标记（token）。


### 使用模型

我们可以像使用标记器一样下载预训练模型。🤗 Transformers 提供了一个 `AutoModel` 类，该类还具有 `from_pretrained()` 方法：


In [11]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


Transformers 模块的矢量输出通常较大。它通常有三个维度：

* Batch size: 一次处理的序列数（在我们的示例中为 2）。
* Sequence length: 序列的数值表示的长度（在我们的示例中为 16）。
* Hidden size: 每个模型输入的向量维度（在我们的示例中为 768）。


![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)


在此图中，模型由其嵌入层和后续层表示。嵌入层将标记化输入中的每个输入 ID 转换为表示关联标记(token)的向量。后续层使用注意机制操纵这些向量，以生成句子的最终表示。

🤗 Transformers 中有许多不同的体系结构，每种体系结构都是围绕处理特定任务而设计的。以下是一个非详尽的列表：

* \*Model (retrieve the hidden states)
* \*ForCausalLM
* \*ForMaskedLM
* \*ForMultipleChoice
* \*ForQuestionAnswering
* \*ForSequenceClassification
* \*ForTokenClassification
* 以及其他 🤗

对于我们的示例，我们需要一个带有序列分类头的模型（能够将句子分类为肯定或否定）。因此，我们实际上不会使用 `AutoModel` 类，而是使用 `AutoModelForSequenceClassification` ：


In [13]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

现在，如果我们观察输出的形状，维度将低得多：模型头将我们之前看到的高维向量作为输入，并输出包含两个值的向量（每个标签一个）：


In [14]:
print(outputs.logits.shape)

torch.Size([2, 2])


因为我们只有两个句子和两个标签，所以我们从模型中得到的结果是 2 x 2 的形状。


### 对输出进行后处理

我们从模型中得到的输出值本身并不一定有意义。我们来看看, 


In [15]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


我们的模型预测第一句为[-1.5607, 1.6123]，第二句为[ 4.1692, -3.3464]。这些不是概率，而是 `logits` ，即模型最后一层输出的原始非标准化分数。要转换为概率，它们需要经过 `SoftMax` 层（所有 🤗Transformers 模型输出 logits，因为用于训练的损耗函数通常会将最后的激活函数（如 SoftMax）与实际损耗函数（如交叉熵）融合）：


In [16]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


现在我们可以看到，

模型预测

第一句为[0.0402, 0.9598]，

第二句为[0.9995, 0.0005]。

这些是可识别的概率分数。

为了获得每个位置对应的具体含义，我们可以检查模型配置的 id2label 属性


In [17]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

现在我们可以得出结论，该模型预测了以下几点：

第一句：否定：0.0402，肯定：0.9598

第二句：否定：0.9995，肯定：0.0005


我们已经成功地复制了管道的三个步骤：使用标记化器进行预处理、通过模型传递输入以及后处理！现在，让我们花一些时间深入了解这些步骤中的每一步。


## 模型

AutoModel 类及其所有相关项实际上是对库中各种可用模型的简单包装。它是一个聪明的包装器，因为它可以自动猜测检查点的适当模型体系结构，然后用该体系结构实例化模型。

但是，如果您知道要使用的模型类型，则可以使用直接定义其体系结构的类。让我们看看这是如何与 `BERT` 模型一起工作的。


In [2]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

/Volumes/data/study/python/NLP/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a Be

#### 使用 Transformers 模型进行推理


Transformer 模型只能处理数字——标记器(Tokenizer)生成的数字。标记器(Tokenizer)可以将输入转换为适当的框架张量。

标记器(Tokenizer)是 NLP 管道的核心组件之一。它们有一个目的：将文本转换为模型可以处理的数据。模型只能处理数字，因此标记器(Tokenizer)需要将我们的文本输入转换为数字数据。


加载和保存标记器(tokenizer)就像使用模型一样简单。实际上，它基于相同的两种方法： `from_pretrained()` 和 `save_pretrained()` 。这些方法将加载或保存标记器(tokenizer)使用的算法（有点像建筑学(architecture)的模型）以及它的词汇（有点像权重(weights)模型）。


In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [4]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

## 标记器(Toeknizer)

### 编码

将文本翻译成数字被称为编码(encoding). 编码分两步完成：标记化，然后转换为输入 ID。


#### 标记化

标记化过程由标记器(tokenizer)的 `tokenize()` 方法实现：


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


这个标记器(tokenizer)是一个子词标记器(tokenizer)：它对词进行拆分，直到获得可以用其词汇表表示的标记(token)。transformer 就是这种情况，它分为两个标记：trans 和 ##former。


###### 从词符(token)到输入 ID

输入 ID 的转换由标记器(tokenizer)的 `convert_tokens_to_ids()` 方法实现：


In [6]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


### 解码


In [7]:
decoded_string = tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 3014])
print(decoded_string)

Using a Transformer network is simple


## 处理多个序列

### 从Tokenizer得到张量


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])  # 因为模型需要传入一批句子所对应的张量，所以这里我们用一个列表包裹了一下
print("input ids:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

/Volumes/data/study/python/NLP/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


input ids: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


这些不是概率，而是 `logits` ，即模型最后一层输出的原始非标准化分数。要转换为概率，它们需要经过 `SoftMax` 层（所有 🤗Transformers 模型输出 logits，因为用于训练的损耗函数通常会将最后的激活函数（如 SoftMax）与实际损耗函数（如交叉熵）融合）：


In [3]:
import torch

predictions = torch.nn.functional.softmax(output.logits, dim=-1)
print(predictions)

tensor([[0.0037, 0.9963]], grad_fn=<SoftmaxBackward0>)


In [2]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

如果传入多个句子，可能类似于这样：


In [ ]:
batched_ids = [ids, ids]

但是因为张量是矩形，如果我们传入的句子长度不一样，需要填充输入后才能传入模型。


In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]  # 这个列表是不能直接转换为张量的。因为列表的长度不一致，不是矩形

了解决这个问题，我们将使用填充使张量具有矩形。Padding通过在值较少的句子中添加一个名为Padding token的特殊单词来确保我们所有的句子长度相同。


In [4]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


In [8]:
tokenizer.pad_token_id

0

我们批处理预测中的logits有点问题：第二行应该与第二句的logits相同，但我们得到了完全不同的值！

这是因为Transformer模型的关键特性是关注层, 它允许模型在处理序列时关注序列中的特定部分。我们需要告诉这些注意层忽略填充标记。这是通过使用 `attention mask` 来实现的。


#### Attention mask

Attention masks是与输入ID张量形状完全相同的张量，用0和1填充：1s表示应注意相应的标记，0s表示不应注意相应的标记（即，模型的注意力层应忽略它们）。


In [9]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids),
                attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


这样我们就得到了正确的结果，与上面第二句话的logits相同。


## 使用 Transformers API 高级函数简化操作

🤗 Transformers API可以通过一个高级函数为我们处理所有这些，我们将在这里深入讨论。当你直接在句子上调用标记器时，你会得到准备通过模型传递的输入


In [10]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

这里， `model_inputs` 变量包含模型良好运行所需的一切。对于DistilBERT，它包括输入 ID和注意力掩码( `attention mask` )。

它还一次处理多个序列，并且API没有任何变化：


In [11]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

它还一次它可以根据几个目标进行填充：


In [12]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

它还可以截断序列:


In [13]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

标记器对象可以处理到特定框架张量的转换，然后可以直接发送到模型。"pt"返回 `PyTorch` 张量，"tf"返回 `TensorFlow` 张量，"np"返回 `NumPy` 数组：


In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

#### 完整实例


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True,
                   truncation=True, return_tensors="pt")
output = model(**tokens)

In [19]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [18]:
predictions = torch.nn.functional.softmax(output.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [5.3534e-04, 9.9946e-01]], grad_fn=<SoftmaxBackward0>)
